In [43]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
import openai

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [21]:
llm=OpenAI(temperature=0)
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [80]:

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)

In [81]:
agent_executor.run("Describe the RWA_122022 table")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: RWA_122022, RWA_032023
Thought: I should query the schema of the RWA_122022 table
Action: sql_db_schema
Action Input: RWA_122022
Observation: 
CREATE TABLE "RWA_122022" (
	"CLIENT_NUMBER" TEXT, 
	"ACCOUNT_NUMBER" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"SEGMENT" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_MORTGAGE" TEXT, 
	"F_REVOLVING" TEXT, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
3 rows from RWA_122022 table:
CLIENT_NUMBER	ACCOUNT_NUMBER	PRODUCT	STATUS	SEGMENT	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_MORTGAGE	F_REVOLVING	F_LARGE_FIN	RW	RWA
sfman997ux8036	1324710203qiph20	CREDIT_CARD	Performing	Retail	0.0	18728.13876	18728.13876	11004.65713	0.0	0.0246518	0.3092	42.58029755	1.15616438	N	Y	N	0.245013691	2696.291657
onvse705c

'The RWA_122022 table contains the following columns: CLIENT_NUMBER, ACCOUNT_NUMBER, PRODUCT, STATUS, SEGMENT, EXPOSURE_DRAWN, EXPOSURE_UNDRAWN, EXPOSURE, EAD, EAD_COLL, PD, LGD, SIZE, MATURITY, F_MORTGAGE, F_REVOLVING, F_LARGE_FIN, RW, and RWA.'

In [32]:
agent_executor.run("What is the total RWA and EAD per SEGMENT in the RWA_122022 table")

'The total RWA and EAD per SEGMENT in the RWA_122022 table are Corporate: 85075812.12498187, 156657332.34921697 and Retail: 37806680.90001285, 165698955.81288758.'

In [38]:
agent_executor.run("What is the total EXPOSURE of all clients that changed their STATUS from Performing to Non Performing between RWA_122022 table and RWA_032023 table")

'The total EXPOSURE of all clients that changed their STATUS from Performing to Non Performing between RWA_122022 table and RWA_032023 table is 1,845,945.9'

In [34]:

agent_executor.run("What is the EXPOSURE of Client 'sfman997ux8036' in RWA_122022 table")

"The exposure of Client 'sfman997ux8036' in RWA_122022 table is 18728.13876."

In [62]:
client = agent_executor.run("Read all the information for Client 'sfman997ux8036' in RWA_122022 table and output it in json format")

In [63]:
client

'[{"CLIENT_NUMBER": "sfman997ux8036", "ACCOUNT_NUMBER": "1324710203qiph20", "PRODUCT": "CREDIT_CARD", "STATUS": "Performing", "SEGMENT": "Retail", "EXPOSURE_DRAWN": 0.0, "EXPOSURE_UNDRAWN": 18728.13876, "EXPOSURE": 18728.13876, "EAD": 11004.65713, "EAD_COLL": 0.0, "PD": 0.0246518, "LGD": 0.3092, "SIZE": 42.58029755, "MATURITY": 1.15616438, "F_MORTGAGE": "N", "F_REVOLVING": "Y", "F_LARGE_FIN": "N", "RW": 0.245013691, "RWA": 2696.291657}]'

#### Function calling

In [47]:
from rw_formula import RW_Calc

In [54]:
# prompt = f"""
# For a client with information in the ClientInfo below calculate the RW if the PD doubles.

# ClientInfo: ```{client}```
# """


#### Attempt 1

In [135]:
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import Tool

from rw_formula import RW_Calc

In [136]:
llm = OpenAI(temperature=0)

prompt = PromptTemplate(
    input_variables=['client'],
    template="what is the RW if the PD of {client} doubles"
)

# Load the tool configs that are needed.
llm_rw_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

RW = Tool(
    name="risk weight calculator",
    func=RW_Calc,
    description=
    """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
)

tools = [RW]

# Construct the react agent type.
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [137]:
agent.run(f"for the client information in {client} what is the RW if PD doubles")



> Entering new  chain...
 I need to calculate the risk weight
Action: risk weight calculator
Action Input: Segment - Retail, PD - 0.0493036, LGD - 0.3092, m - 1.15616438, Large_Fin - N, size - 42.58029755, mortgage - N, revolving - Y

TypeError: RW_Calc() missing 7 required positional arguments: 'PD', 'LGD', 'm', 'Large_Fin', 'size', 'mortgage', and 'revolving'

#### Attempt 2

In [138]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [139]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="risk weight calculator",
        func=RW_Calc,
        description=
        """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
    ),    
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [140]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [141]:
agent.run("Read all the information for Client 'sfman997ux8036' in RWA_122022 table and then calculate their RW if PD doubles")



> Entering new  chain...
I need to query the RWA_122022 table to get the information for Client 'sfman997ux8036'. Then I can use the risk weight calculator to calculate their RW if PD doubles.
Action: rwa_db
Action Input: SELECT * FROM RWA_122022 WHERE Client = 'sfman997ux8036'

> Entering new  chain...
SELECT * FROM RWA_122022 WHERE Client = 'sfman997ux8036'
SQLQuery:SELECT * FROM RWA_122022 WHERE CLIENT_NUMBER = 'sfman997ux8036'
SQLResult: [('sfman997ux8036', '1324710203qiph20', 'CREDIT_CARD', 'Performing', 'Retail', 0.0, 18728.13876, 18728.13876, 11004.65713, 0.0, 0.0246518, 0.3092, 42.58029755, 1.15616438, 'N', 'Y', 'N', 0.245013691, 2696.291657)]
Answer:CLIENT_NUMBER: sfman997ux8036
ACCOUNT_NUMBER: 1324710203qiph20
PRODUCT: CREDIT_CARD
STATUS: Performing
SEGMENT: Retail
EXPOSURE_DRAWN: 0.0
EXPOSURE_UNDRAWN: 18728.13876
EXPOSURE: 18728.13876
EAD: 11004.65713
EAD_COLL: 0.0
PD: 0.0246518
LGD: 0.3092
SIZE: 42.58029755
MATURITY: 1.15616438
F_MORTGAGE: N
F_REVOLVING: Y
F_LARGE_FIN: N


TypeError: RW_Calc() missing 7 required positional arguments: 'PD', 'LGD', 'm', 'Large_Fin', 'size', 'mortgage', and 'revolving'

#### Attempt 3

In [113]:
prompt = f"""
For a client with information in the ClientInfo below calculate with RW_Calc function and return the RW if the PD doubles.

ClientInfo: ```{client}```
"""

In [117]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": prompt}],
    functions=[
    {
        "name": 'RW_Calc',
        "description": "Calculate RW",
        "parameters": {
            "type": "object",
            "properties": {
                "Segment": {
                    # "type": "text",
                    "description": "Possible values are 'Bank', 'Corporate', and 'Retail'",
                },
                "PD": {
                    # "type": "real", 
                    "description": "Probability of Default",
                },
                "LGD": {
                    # "type": "real", 
                    "description": "Loss Given Default",
                },
                "m": {
                    # "type": "real", 
                    "description": "Remaining maturity of the loan in years",
                },
                 "Large_Fin": {
                    # "type": "text", 
                    "description": "If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'",
                },
                 "size": {
                    # "type": "real", 
                    "description": "size of the client in MEUR, usually this is the client's turnover",
                },
                "mortgage": {
                    # "type": "text", 
                    "description": "If 'Y' the exposure is a mortgage loan, otherwise 'N'",
                 },
                "revolving": {
                    # "type": "text", 
                    "description": "If 'Y' the exposure is a revolving loan, otherwise 'N'",
                 },

            },
        # "required": ['RW', 'Segment', 'PD', 'LGD', 'm', 'Large_Fin', 'size', 'mortgage', 'revolving'],
        "required": ['RW'],
        },
    }
],
function_call="auto",
)


In [118]:
completion.choices[0].message

<OpenAIObject at 0x7f35aa0e1630> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "RW_Calc",
    "arguments": "{\n  \"Segment\": \"Retail\",\n  \"PD\": 0.0246518,\n  \"LGD\": 0.3092,\n  \"m\": 1.15616438,\n  \"Large_Fin\": \"N\",\n  \"size\": 42.58029755,\n  \"mortgage\": \"N\",\n  \"revolving\": \"Y\"\n}"
  }
}

In [116]:
import json

reply_content = completion.choices[0].message

funcs = reply_content.to_dict()['function_call']['arguments']
funcs = json.loads(funcs)
print(funcs)

{'Segment': 'Retail', 'PD': 0.0246518, 'LGD': 0.3092, 'm': 1.15616438, 'Large_Fin': 'N', 'size': 42.58029755, 'mortgage': 'N', 'revolving': 'Y'}


#### Attempt 4

In [119]:
prompt = f"""
For a client with information in the ClientInfo below calculate with RW_Calc function and return the RW if the PD doubles.

ClientInfo: ```{client}```
"""
messages = [
    {"role": "user", "content": prompt},
]


def get_completion(messages):
    response = openai.ChatCompletion.create(
        model= 'gpt-3.5-turbo-0613',
        messages=messages,
        functions=[
            {
                "name": 'RW_Calc',
                "description": "Calculate RW",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "Segment": {
                            # "type": "text",
                            "description": "Possible values are 'Bank', 'Corporate', and 'Retail'",
                        },
                        "PD": {
                            # "type": "real", 
                            "description": "Probability of Default",
                        },
                        "LGD": {
                            # "type": "real", 
                            "description": "Loss Given Default",
                        },
                        "m": {
                            # "type": "real", 
                            "description": "Remaining maturity of the loan in years",
                        },
                        "Large_Fin": {
                            # "type": "text", 
                            "description": "If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'",
                        },
                        "size": {
                            # "type": "real", 
                            "description": "size of the client in MEUR, usually this is the client's turnover",
                        },
                        "mortgage": {
                            # "type": "text", 
                            "description": "If 'Y' the exposure is a mortgage loan, otherwise 'N'",
                        },
                        "revolving": {
                            # "type": "text", 
                            "description": "If 'Y' the exposure is a revolving loan, otherwise 'N'",
                        },
                    },
                "required": ['RW', 'Segment', 'PD', 'LGD', 'm', 'Large_Fin', 'size', 'mortgage', 'revolving'],        
            },
            },            
        ],
        temperature=0,
    )

    return response

In [121]:
response = get_completion(messages)

In [122]:
response

<OpenAIObject chat.completion id=chatcmpl-7fm6E3DndIKRJSNrt6J1WMLaUNTp0 at 0x7f35aa0abc70> JSON: {
  "id": "chatcmpl-7fm6E3DndIKRJSNrt6J1WMLaUNTp0",
  "object": "chat.completion",
  "created": 1690190698,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "RW_Calc",
          "arguments": "{\n  \"Segment\": \"Retail\",\n  \"PD\": 0.0246518,\n  \"LGD\": 0.3092,\n  \"m\": 1.15616438,\n  \"Large_Fin\": \"N\",\n  \"size\": 42.58029755,\n  \"mortgage\": \"N\",\n  \"revolving\": \"Y\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 407,
    "completion_tokens": 86,
    "total_tokens": 493
  }
}

#### Attempt 5

In [123]:
def get_completion(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=[{
            "name": "RW_Calc",
            "description": "Calculate RW for given inputs.",
            "parameters": { "type": "object", "properties": {} },
        }],
        temperature=0,
    )

    return response

In [133]:
prompt = f"""
For a client with information in the ClientInfo below calculate with RW_Calc function and return the RW if the PD doubles.

ClientInfo: ```{client}```
"""
messages = [
    {"role": "user", "content": prompt},
]

response = get_completion(messages)

result = RW_Calc

In [130]:
result

<function rw_formula.RW_Calc(Segment, PD, LGD, m, Large_Fin, size, mortgage, revolving)>